In [1]:
import numpy as np
from scipy.sparse import csr_matrix, find
from scipy.spatial import cKDTree
from tqdm import tqdm_notebook as tqdm

from datasets.graph import draw_sample
import torch


In [2]:
model_fname = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/output_HGC_pion8nn_siminfo/checkpoints/model_checkpoint_EdgeNetWithCategories_264403_5b5c05404f_sameasy2006.best.pth.tar'
from models.EdgeNetWithCategories import EdgeNetWithCategories
mdl = EdgeNetWithCategories(input_dim=5, hidden_dim=64, output_dim=4, n_iters=6).to('cuda:0')
mdl.load_state_dict(torch.load(model_fname)['model'])
mdl.eval()


EdgeNetWithCategories(
  (inputnet): Sequential(
    (0): Linear(in_features=5, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): Tanh()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): Tanh()
  )
  (edgenetwork): Sequential(
    (0): Linear(in_features=768, out_features=128, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=128, out_features=4, bias=True)
    (5): LogSoftmax()
  )
  (nodenetwork0): EdgeConv(nn=Sequential(
    (0): Linear(in_features=138, out_features=101, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=101, out_features=64, bias=True)
    (3): ELU(alpha=1.0)
  ))
  (nodenetwork1): EdgeConv(nn=Sequential(
    (0): Linear(in_features=138, out_features=101, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=101, out_features=64, bias=True)
    (3): ELU(alpha=1.0

In [3]:
from unionfind import UnionFind 
def cluster_points(X,out):
#test_fname = '/home/lagray/training_data/single_taus/test/processed/data_40.pt'from unionfind import UnionFind
    finder_had = UnionFind(X.shape[0])
    finder_pho = UnionFind(X.shape[0])
    finder_mip = UnionFind(X.shape[0])

#    for i in tqdm(range(index.shape[0])):
    for i in (range(index.shape[0])):
        if out[i] == 1:
            finder_had.union(index[i,0], index[i,1])
        if out[i] == 2:
            finder_pho.union(index[i,0], index[i,1])
        if out[i] == 3:
            finder_mip.union(index[i,0], index[i,1])

    had_roots = np.array([finder_had.find(i) for i in range(X.shape[0])], dtype=np.uint32)
    pho_roots = np.array([finder_pho.find(i) for i in range(X.shape[0])], dtype=np.uint32)
    mip_roots = np.array([finder_mip.find(i) for i in range(X.shape[0])], dtype=np.uint32)


    had_clusters = np.unique(had_roots, return_inverse=True, return_counts=True)
    pho_clusters = np.unique(pho_roots, return_inverse=True, return_counts=True)
    mip_clusters = np.unique(mip_roots, return_inverse=True, return_counts=True)

    hads = had_clusters[0][np.where(had_clusters[2] > 4)]
    ems = pho_clusters[0][np.where(pho_clusters[2] > 4)]
    mips = mip_clusters[0][np.where(mip_clusters[2] > 4)]

    had_clusters_sel = {i: np.where(had_roots == had)[0] for i, had in enumerate(hads)}
    em_clusters_sel = {i: np.where(pho_roots == em)[0] for i, em in enumerate(ems)}
    mip_clusters_sel = {i: np.where(mip_roots == mip)[0] for i, mip in enumerate(mips)}
    
    return had_clusters_sel,em_clusters_sel,mip_clusters_sel

In [6]:
import glob
flist = [filepath for filepath in glob.iglob(r'/home/sameasy2006/npz_hgcal_pionaddsiminfo_all_k8/pion_hgctup_0to1000/processed/data_*.pt')]
print (len(flist))
#flist = flist[13900:14400] 
#flist = flist[0:2000]
#print(flist)

14146


In [7]:
import torch_geometric
import torch
datacls = []
for filename in tqdm(flist):
#    print(filename)
    data = torch.load(filename).to('cuda:0')
#    X = data.x.detach().numpy()
    X = data.x.cpu().numpy()
    pred_edges = mdl(data).detach()
#    pred_edges_np = pred_edges.numpy()
    pred_edges_np = pred_edges.cpu().numpy()
#    index = data.edge_index.numpy().T
    index = data.edge_index.cpu().numpy().T
    Ro = index[:,0]
    Ri = index[:,1]
#    y = data.y.numpy()
    y = data.y.cpu().numpy()
    out =np.argmax(pred_edges_np,axis=-1)
    
#    print (np.unique(y)   ,  X[np.unique(Ro[y==1])][:,4].sum())
    z  = data.z.cpu().numpy()
    true_y = z[0]
#    print (true_y)
    if len(cluster_points(X,out)[0].keys())  > 1 :
        continue
    #print (len(cluster_points(X,out)[0].keys()))
    datacls.extend([torch_geometric.data.Data(x = torch.tensor(X[clus]), y=torch.tensor([true_y])) for clus in cluster_points(X,out)[0].values()])

    
    
#datas = torch_geometric.data.DataLoader(datacls, batch_size = 60)    

/home/sameasy2006/miniconda3/envs/mlgnn2/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [23]:
#datacls

In [8]:
processed_dir = '/home/sameasy2006/npz_hgcal_pionaddsiminfo_all_k8/pion_hgctup_0to1000/clusters/'
import os
import os.path as osp
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
#processed_dir = '/home/sameasy2006/npz_hgcal_k8/pion_hgctup_0to1000/clusters/'
for i in tqdm(range(len(datacls))):
    torch.save(datacls[i], osp.join(processed_dir, 'data_{}.pt'.format(i)))

/home/sameasy2006/miniconda3/envs/mlgnn2/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [4]:
import glob
raw_dir= '/home/sameasy2006/npz_hgcal_k8/pion_hgctup_0to1000/clusters/'
fnamelist = [filepath for filepath in glob.glob(raw_dir+'data_*.pt')]
data_list = []
for i in tqdm(fnamelist):
    data_list.append(torch.load(i))

/home/sameasy2006/miniconda3/envs/mlgnn/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [6]:
import torch_geometric
loader = torch_geometric.data.DataLoader(data_list, batch_size=10)

In [10]:
import os.path as osp

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from tqdm import tqdm_notebook as tqdm


from models.DynamicReductionNetwork import DynamicReductionNetwork

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drn = DynamicReductionNetwork()
        
    def forward(self, data):
        logits = self.drn(data)
        return F.softmax(logits)



In [11]:
device = torch.device('cpu')#('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [12]:
model.train()
#def train(epoch):
#    model.train()
#
#    if epoch == 16:
#        for param_group in optimizer.param_groups:
#            param_group['lr'] = 0.005
#    
#    if epoch == 32:
#        for param_group in optimizer.param_groups:
#            param_group['lr'] = 0.0001
#
#    if epoch == 48:
#        for param_group in optimizer.param_groups:
#            param_group['lr'] = 0.00001

for data in (loader):
        data = data.to(device)        
        print (data.batch)
        print (model(data))
        print (data.y)
#        optimizer.zero_grad()
 #       result = model(data)
        F.nll_loss(result, data.y).backward()
        print (F.nll_loss(result, data.y).backward())
        break
#        mask = (data.x > 0.).squeeze()
#        data.x = torch.cat([data.x, data.pos], dim=-1)
#        data.x = data.x[mask,:]
        #print(data.x)
#        data.pos = data.pos[mask,:]
#        data.batch = data.batch[mask.squeeze()]
#        optimizer.zero_grad()
#        result = model(data)
        F.nll_loss(result, data.y).backward()
        #print(torch.unique(torch.argmax(result, dim=-1)))
        #print(torch.unique(data.y))
#        optimizer.step()
#        print (F.nll_loss(result, data.y).backward())

tensor([0, 0, 0,  ..., 9, 9, 9])


NameError: name 'output' is not defined